# Modified to create an animation showing analysis method -- Event finder & raw data checker

This notebook will find competitive cellular "events" in the simplest definition (i.e. a loser cell apoptosis) and return information about the spatiotemporal distrubition of counterpart competitive events (i.e. winner cell mitosis)

Contents:

- Load modules
- Load cell finding functions
- Set experiment data path
- Load image data
- Load tracking data
- Apply necessary coordinate shift for viewer
- Isolate one track of interest (target track)
- Find corresponding tracks/events of interest within a given spatiotemporal range (E.g. if target track is Scr apoptosis, then find all nearby wild-type mitosis)
- Set points and regions of interest for highlighting in Napari viewer(now set as functions so this bit is automatic
- Launch Napari image viewer if desired


In [1]:
import sys
sys.path.append('../')

import napari
import btrack
import numpy as np
from skimage.io import imread
import os
print("Napari version no.:", napari.__version__)
print("btrack version no.:", btrack.__version__)
from btrack.utils import import_HDF, import_JSON, tracks_to_napari
from tqdm.notebook import tnrange, tqdm
import matplotlib.pyplot as plt
import tools
from datetime import datetime
from napari_animation import AnimationWidget

Napari version no.: 0.4.7
btrack version no.: 0.4.1


### Set experiment data path 

In [2]:
root_path = '/home/nathan/data/kraken/h2b/giulia/GV0807'  ## this overwrites input option for ease 
gfp_path = os.path.join(root_path, 'Pos3/stacks/gfp.tif')
rfp_path = os.path.join(root_path, 'Pos3/stacks/rfp.tif')
bf_path = os.path.join(root_path, 'Pos3/stacks/bf.tif')
tracks_path = os.path.join(root_path, 'Pos3/Pos3_aligned/HDF/segmented.hdf5')

### Load image data

In [3]:
gfp = imread(gfp_path)

In [4]:
rfp = imread(rfp_path)

In [5]:
bf = imread(bf_path) #3 large 1000+ frame stacks is usually too much for memory to process on 16gb ram

### Load tracking data

In [6]:
target_trackwith btrack.dataio.HDF5FileHandler(tracks_path, 'r', obj_type = "obj_type_1") as hdf:
    wt_tracks = hdf.tracks
with btrack.dataio.HDF5FileHandler(tracks_path, 'r', obj_type = "obj_type_2") as hdf:
    scr_tracks = hdf.tracks

## this method casues problems as the viewer for tracks doesnt like negative numbers
#wt_tracks, scr_tracks, all_tracks = tools.load_tracking_data(tracks_path)

print("Tracks loaded")

### finding coord range of aligned images, coords switched already ## need to sort out the order of try excepts
try:
    align_x_range, align_y_range = gfp.shape[2], gfp.shape[1]    
except:
    print()
    try:
        align_x_range, align_y_range = bf.shape[2], bf.shape[1]
    except:
        print("Error: no image data loaded to map tracks to")
try: 
    align_x_range, align_y_range = rfp.shape[2], rfp.shape[1]
except:
    print()
    try:
        align_x_range, align_y_range = bf.shape[2], bf.shape[1]
    except:
        print("Error: no image data loaded to map tracks to")


### finding maximum extent of tracking coords
tracks_x_range = round(max([max(track.x) for track in wt_tracks]))
tracks_y_range = round(max([max(track.y) for track in wt_tracks])) + 2 ## sort this lazy hack out later

### coord switch
tmp = tracks_y_range
tracks_y_range = tracks_x_range
tracks_x_range = tmp

print("tracks range:", (tracks_x_range), (tracks_y_range))
print("aligned image range:", (align_x_range), (align_y_range))

shift_x = int((align_x_range - tracks_x_range)/2)
shift_y = int((align_y_range - tracks_y_range)/2)

print("shift in x and y:", shift_x, shift_y)

wt_data, properties, graph = tracks_to_napari(wt_tracks, ndim = 2)
scr_data, properties, graph = tracks_to_napari(scr_tracks, ndim = 2)

tmp = wt_data[:,2].copy() ## copy the true_y coord
wt_data[:,2] = wt_data[:,3]  ##assign the old_y coord as the true_x
wt_data[:,3] = tmp ## assign the old_x as true_y

wt_data[:,2] += shift_y ## TRUE_Y (vertical axis)
wt_data[:,3] += shift_x ## TRUE_X (horizontal axis)

tmp = scr_data[:,2].copy()
scr_data[:,2] = scr_data[:,3]
scr_data[:,3] = tmp

scr_data[:,2] += shift_y ## TRUE_Y (vertical axis)
scr_data[:,3] += shift_x ## TRUE_X (horizontal axis)

print("coordinate shift applied")

[INFO][2021/05/07 04:44:22 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5...
[INFO][2021/05/07 04:44:22 PM] Loading tracks/obj_type_1
[INFO][2021/05/07 04:44:25 PM] Loading objects/obj_type_1 (408973, 5) (388394 filtered: area>=100)
[INFO][2021/05/07 04:44:30 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5
[INFO][2021/05/07 04:44:30 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5...
[INFO][2021/05/07 04:44:30 PM] Loading tracks/obj_type_2
[INFO][2021/05/07 04:44:30 PM] Loading objects/obj_type_2 (12115, 5) (8894 filtered: area>=100)
[INFO][2021/05/07 04:44:30 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5


Tracks loaded
tracks range: 1600 1200
aligned image range: 1739 1377
shift in x and y: 69 88
coordinate shift applied


### Isolate one scribble track of interest

In [7]:
## isolate one target scribble track of interest
print("input scribble track of interest ID")
cell_ID = 17#int(input())
target_track = [scr_track for scr_track in scr_tracks if scr_track.ID == cell_ID][0]
#apop_time, apop_index = find_apoptosis_time(target_track, index = False), find_apoptosis_time(target_track, index = True)  
target_track

input scribble track of interest ID


OrderedDict([('ID', 17), ('t', [354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742]), ('x', [592.0890502929688, 599.7708129882812, 597.2949829101562, 595.1409301757812, 596.6591186523438, 598.0703125, 601.8123168945312, 606.079833984375, 608.4224853515625, 610.5025024414062, 609.2481079101562, 609.672119140625, 609.0426025390625, 609.945556640625, 609.80712890625, 609.967529296875, 610.3758544921875, 608.7223510742188, 608.5475463867188, 607.1779174804688, 606.2504272460938, 605.3333129882812, 606.1070556640625, 605.3349609375, 608.6104736328125, 609.324951171875, 607.1929321289062, 608.331298828125, 605.822998046875, 606.0330810546875, 607.8743896484375, 607.95068359375, 608.8442993164062, 609.58642578125, 610.369140625, 611.337890625, 611.572265625, 615.2579345703125, 614.7274169921875, 615.192626953125, 614.005126953125, 614.6265869140625, 616.01708984375, 616.0947875976562, 618.281005859375, 618.39453125, 618.39501953125, 618.4774169921875, 617.001220703125, 617.0227661132812, 617.0206298828125, 616.1651000976562, 615.47900390625, 617.8047485351562, 616.5301513671875, 615.956787109375, 616.1814575195312, 614.3001098632812, 614.8740234375, 613.3399047851562, 613.8902587890625, 610.8990478515625, 608.5623779296875, 606.3654174804688, 602.3408813476562, 600.3064575195312, 598.9340209960938, 595.8876342773438, 593.6514282226562, 596.6903076171875, 598.8035888671875, 604.9063720703125, 605.813720703125, 605.51708984375, 607.9710693359375, 606.9283447265625, 607.6414184570312, 606.8410034179688, 606.6137084960938, 610.6298828125, 611.1257934570312, 612.5712890625, 614.0626831054688, 612.5009765625, 614.7109375, 614.8167114257812, 612.7708740234375, 612.6990966796875, 609.9771728515625, 607.38818359375, 608.2338256835938, 604.4841918945312, 604.7588500976562, 601.2293090820312, 600.881591796875, 598.3723754882812, 598.7335205078125, 596.8101806640625, 597.46240234375, 597.0515747070312, 594.7959594726562, 593.4949340820312, 593.7627563476562, 591.3768310546875, 592.331787109375, 589.9038696289062, 590.0537109375, 590.315673828125, 588.9689331054688, 589.8568115234375, 590.232360

In [13]:
## isolate one target wt track of interest
print("input scribble track of interest ID")
cell_ID = 312#int(input())
target_track = [wt_track for wt_track in wt_tracks if wt_track.ID == cell_ID][0]
#apop_time, apop_index = find_apoptosis_time(target_track, index = False), find_apoptosis_time(target_track, index = True)  
target_track

input scribble track of interest ID


OrderedDict([('ID', 312), ('t', [420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637]), ('x', [646.0625, 650.89794921875, 652.1168212890625, 649.0828857421875, 644.5145874023438, 645.8594970703125, 648.1783447265625, 647.3180541992188, 651.2125854492188, 642.1560668945312, 642.8240356445312, 644.1619262695312, 643.7092895507812, 643.091064453125, 644.162353515625, 644.9143676757812, 645.679931640625, 646.628662109375, 647.1873779296875, 648.0477294921875, 648.5469970703125, 648.318359375, 646.8916625976562, 646.0316162109375, 646.7252807617188, 646.27685546875, 646.2692260742188, 646.195068359375, 647.7115478515625, 647.3441772460938, 647.058837890625, 646.2268676757812, 645.4510498046875, 644.4058227539062, 644.7005004882812, 645.4896240234375, 645.3994750976562, 645.0051879882812, 645.007080078125, 644.617431640625, 645.0, 644.935546875, 645.0209350585938, 645.2470092773438, 645.055908203125, 643.893310546875, 644.3477172851562, 643.824462890625, 643.8193359375, 645.0450439453125, 645.1906127929688, 644.5341186523438, 644.3072509765625, 643.5134887695312, 644.0081787109375, 644.38720703125, 643.9351196289062, 644.1485595703125, 644.3231811523438, 644.20654296875, 645.0054931640625, 644.4389038085938, 644.0189819335938, 644.1150512695312, 645.0281982421875, 644.871337890625, 644.0686645507812, 642.9828491210938, 643.0929565429688, 643.2111206054688, 642.4256591796875, 641.7366943359375, 643.0880737304688, 642.3936157226562, 643.074951171875, 643.1270141601562, 643.5983276367188, 644.6074829101562, 645.5919189453125, 646.632080078125, 646.864013671875, 647.3809814453125, 648.2380981445312, 647.7116088867188, 646.706787109375, 648.67333984375, 649.5433349609375, 650.3678588867188, 649.5786743164062, 649.3416137695312, 651.0206298828125, 651.663818359375, 652.9420776367188, 653.1646118164062, 653.232177734375, 653.7180786132812, 655.049560546875, 656.7786254882812, 658.3327026367188, 657.8318481445312, 659.592041015625, 660.7333374023438, 660.7978515625, 660.812744140625, 662.4298706054688, 663.1351928710938, 663.567138671875, 665.091064453125, 664.096435546875, 663.3058471679688, 661.6126098632812, 662.0640869140625, 659.4954223632812, 658.4885864257812, 657.5652465820312, 657.2852172851562, 655.4695434570312, 653.5086669921875, 651.9880981445312, 651.0084838867188, 650.7452392578125, 650.4052734375, 649.8710327148438, 650.4389038085938, 650.4766845703125, 651.1748046875, 652.4146118164062, 653.48486328125, 653.9811401367188, 655.071533203125, 656.8113403320312, 657.9636840820312, 658.8108520507812, 659.0794067382812, 661.5208129882812, 664.1384887695312, 664.2138671875, 666.7479248046875, 667.3058471679688, 668.5858154296875, 668.61083984375, 668.20068359375, 667.6700439453125, 667.138916015625, 668.70361328125, 668.07177734375, 667.7307739257812, 667.3670654296875, 667.3870849609375, 668.012451171875, 667.6421508789062, 666.930908203125, 667.830810546875, 668.4102783203125, 668.0106811523438, 668.519775390625

### Functions to display local neighbourhood for cells of interest

In [18]:
def plot_mitoses(cell_type, cell_ID, radius, delta_t): ## this function plots mitosis events into the napari viewer
    if cell_type.lower() == 'scr':
        target_track = [track for track in scr_tracks if track.ID == cell_ID][0]
    else:
        target_track = [track for track in wt_tracks if track.ID == cell_ID][0]
    apop_time, apop_index = find_apoptosis_time(target_track, index = False), find_apoptosis_time(target_track, index = True)  
    apop_event = target_track.t[apop_index], target_track.x[apop_index]+shift_y, target_track.y[apop_index]+shift_x ## with transposed shift
    wt_tracks_in_radius, wt_mitosis_in_radius = find_nearby_wt_mitosis(target_track, delta_t, radius)
    t_m, x_m, y_m = np.zeros(len(wt_mitosis_in_radius)), np.zeros(len(wt_mitosis_in_radius)), np.zeros(len(wt_mitosis_in_radius))
    mito_events = np.zeros((len(wt_mitosis_in_radius), 3)) ## 3 because of the 3 cartesian coords 
    for i, wt_mitosis in enumerate(wt_mitosis_in_radius): ## this now assumes that the mitosis time point of relevance isnt the last frame of track but the time at delta_t, need to bolster definition of mitosis
        mito_index = [j for j, k in enumerate(wt_mitosis.t) if k == apop_event[0]+delta_t][0] ### [0] bc first item of list comprehension
        t_m[i], x_m[i], y_m[i] = wt_mitosis.t[mito_index], wt_mitosis.x[mito_index]+shift_y, wt_mitosis.y[mito_index]+shift_x ## plus transposed coordinate shift
        mito_events[i] = t_m[i], x_m[i], y_m[i]
    return viewer.add_points(mito_events, name = "Mitosis events", symbol = "cross", face_color = 'pink')

def plot_target_track(cell_type, cell_ID):
    if cell_type.lower() == 'scr':
        target_track = [track for track in scr_tracks if track.ID == cell_ID][0]
    else:
        target_track = [track for track in wt_tracks if track.ID == cell_ID][0]
    target_track_loc = [(target_track.t[i], target_track.x[i]+shift_y, target_track.y[i]+shift_x) for i in range(len(target_track.t))]
    return viewer.add_points(target_track_loc, name = "Track of interest", size = 40, symbol = 'o', face_color = "transparent", edge_color = 'cyan', edge_width = 2)    

def plot_stationary_apoptosis_point(cell_type, cell_ID): ## this function plots apoptotic event and surrounding local environment scope (determined by radius)
    if cell_type.lower() == 'scr':
        target_track = [track for track in scr_tracks if track.ID == cell_ID][0]
    else:
        target_track = [track for track in wt_tracks if track.ID == cell_ID][0]
    apop_time, apop_index = find_apoptosis_time(target_track, index = False), find_apoptosis_time(target_track, index = True)  
    apop_event = [(t, target_track.x[apop_index]+shift_y, target_track.y[apop_index]+shift_x) for t in range(len(gfp))] ## marker for apoptosis over all frames
    return viewer.add_points(apop_event, name = "Stastionary apoptosis point", size = 40, symbol = 'o', face_color = "transparent", edge_color = 'cyan', edge_width = 2)

def plot_stationary_apop_radius(cell_type, cell_ID, radius, delta_t, inner_radius):
    if cell_type.lower() == 'scr':
        target_track = [track for track in scr_tracks if track.ID == cell_ID][0]
    else:
        target_track = [track for track in wt_tracks if track.ID == cell_ID][0]
    apop_time, apop_index = find_apoptosis_time(target_track, index = False), find_apoptosis_time(target_track, index = True)  
    apop_event = target_track.t[apop_index], target_track.x[apop_index]+shift_y, target_track.y[apop_index]+shift_x ## with transposed shift, just for the frame of apoptosis
    outer_radial_bin = [tuple(((apop_event[0]+t, apop_event[1]-radius, apop_event[2]-radius), 
                               (apop_event[0]+t, apop_event[1]+radius, apop_event[2]-radius), 
                               (apop_event[0]+t, apop_event[1]+radius, apop_event[2]+radius), 
                               (apop_event[0]+t, apop_event[1]-radius, apop_event[2]+radius))) 
                                for t in range(-abs(delta_t), +abs(delta_t)+1)]
    if inner_radius > 0:
        inner_radial_bin = [tuple(((apop_event[0]+t, apop_event[1]-inner_radius, apop_event[2]-inner_radius), 
                                   (apop_event[0]+t, apop_event[1]+inner_radius, apop_event[2]-inner_radius), 
                                   (apop_event[0]+t, apop_event[1]+inner_radius, apop_event[2]+inner_radius), 
                                   (apop_event[0]+t, apop_event[1]-inner_radius, apop_event[2]+inner_radius))) 
                                    for t in range(-abs(delta_t), +abs(delta_t)+1)]
        return viewer.add_shapes(outer_radial_bin,opacity = 1, shape_type = 'ellipse', face_color = 'transparent', edge_color = 'cyan', edge_width = 5, name = 'Radial environment'), viewer.add_shapes(inner_radial_bin, opacity = 1, shape_type = 'ellipse', face_color = 'transparent', edge_color = 'cyan', edge_width = 5, name = 'Inner Radial environment')
    else:
        return viewer.add_shapes(outer_radial_bin, opacity = 1, shape_type = 'ellipse', face_color = 'transparent', edge_color = 'cyan', edge_width = 5, name = 'Radial environment')
    
def plot_radius(cell_type, cell_ID, radius):
    if cell_type.lower() == 'scr':
        target_track = [track for track in scr_tracks if track.ID == cell_ID][0]
    else:
        target_track = [track for track in wt_tracks if track.ID == cell_ID][0]
    radius_shape = [tuple(((t, target_track.x[i]+shift_y-radius, target_track.y[i]+shift_x-radius), 
                   (t, target_track.x[i]+shift_y+radius, target_track.y[i]+shift_x-radius), 
                   (t, target_track.x[i]+shift_y+radius, target_track.y[i]+shift_x+radius), 
                   (t, target_track.x[i]+shift_y-radius, target_track.y[i]+shift_x+radius))) 
                    for i,t in enumerate(range(target_track.t[0], target_track.t[-1]))]
    return viewer.add_shapes(radius_shape, opacity = 1, shape_type = 'ellipse', face_color = 'transparent', edge_color = 'cyan', edge_width = 5, name = 'Radial environment')

def plot_post_track_radius(cell_type, cell_ID, radius):
    if cell_type.lower() == 'scr':
        target_track = [track for track in scr_tracks if track.ID == cell_ID][0]
    else:
        target_track = [track for track in wt_tracks if track.ID == cell_ID][0]
    radius_shape = [tuple(((t, target_track.x[-1]+shift_y-radius, target_track.y[-1]+shift_x-radius), 
                   (t, target_track.x[-1]+shift_y+radius, target_track.y[-1]+shift_x-radius), 
                   (t, target_track.x[-1]+shift_y+radius, target_track.y[-1]+shift_x+radius), 
                   (t, target_track.x[-1]+shift_y-radius, target_track.y[-1]+shift_x+radius))) 
                    for i,t in enumerate(range(target_track.t[-1],len(gfp)))]
    return viewer.add_shapes(radius_shape, opacity = 1, shape_type = 'ellipse', face_color = 'transparent', edge_color = 'cyan', edge_width = 5, name = 'Post-apoptosis radial environment')

def plot_fragmented_track(list_of_IDs): ### not using this below as dont think output is correct
    compiled_frag_track_loc = []
    compiled_frag_radius_loc = []
    for cell_ID in list_of_IDs:
        target_track = [track for track in scr_tracks if track.ID == cell_ID][0]
        #plot_radius(target_track)
        #plot_target_track(target_track)
        radius_loc = plot_frag_radius(target_track)
        compiled_frag_radius_loc+= radius_loc
        target_track_loc = plot_frag_target_track(target_track)
        compiled_frag_track_loc += target_track_loc
    return viewer.add_shapes(compiled_frag_radius_loc, opacity = 1, shape_type = 'ellipse', face_color = 'transparent', edge_color = 'cyan', edge_width = 5, name = 'Radial environment'), viewer.add_points(compiled_frag_track_loc, name = "Track of interest", size = 40, symbol = 'o', face_color = "transparent", edge_color = 'cyan', edge_width = 2)

def plot_frag_target_track(target_track):
    if target_track.ID == 17:
        target_track_loc = [(target_track.t[i], target_track.x[i]+shift_y, target_track.y[i]+shift_x) for i in range(len(target_track.t))]
        return target_track_loc #viewer.add_points(target_track_loc, name = "Track of interest", size = 40, symbol = 'o', face_color = "transparent", edge_color = 'cyan', edge_width = 2)    
    else:
        target_track_loc = [(target_track.t[i], target_track.x[i]+shift_y, target_track.y[i]+shift_x) for i in range(len(target_track.t)) if target_track.t[i]> 742]
        return target_track_loc#viewer.add_points(target_track_loc, name = "Track of interest", size = 40, symbol = 'o', face_color = "transparent", edge_color = 'cyan', edge_width = 2)    

def plot_frag_radius(target_track):
    if target_track.ID ==17:### this if condition is to avoid double plotting radii as fragmented tracks exist at same time
        radius_shape = [tuple(((t, target_track.x[i]+shift_y-radius, target_track.y[i]+shift_x-radius), 
                       (t, target_track.x[i]+shift_y+radius, target_track.y[i]+shift_x-radius), 
                       (t, target_track.x[i]+shift_y+radius, target_track.y[i]+shift_x+radius), 
                       (t, target_track.x[i]+shift_y-radius, target_track.y[i]+shift_x+radius))) 
                        for i,t in enumerate(range(target_track.t[0], target_track.t[-1]))]
        return radius_shape
    else:
        radius_shape = [tuple(((t, target_track.x[i]+shift_y-radius, target_track.y[i]+shift_x-radius), 
                       (t, target_track.x[i]+shift_y+radius, target_track.y[i]+shift_x-radius), 
                       (t, target_track.x[i]+shift_y+radius, target_track.y[i]+shift_x+radius), 
                       (t, target_track.x[i]+shift_y-radius, target_track.y[i]+shift_x+radius))) 
                        for i,t in enumerate(range(target_track.t[0], target_track.t[-1])) if t>741]
        return radius_shape

def plot_radii(cell_type, target_track, radius, num_bins):
    print('This can be very time consuming for >10 bins, consider using single_frame radius')
    radii = range(int(radius/num_bins), radius+int(radius/num_bins), int(radius/num_bins))    
    if cell_type.lower() == 'scr':
        target_track = [track for track in scr_tracks if track.ID == cell_ID][0]
    else:
        target_track = [track for track in wt_tracks if track.ID == cell_ID][0]
    radius_shape = [tuple(((t, target_track.x[i]+shift_y-radius, target_track.y[i]+shift_x-radius), 
                   (t, target_track.x[i]+shift_y+radius, target_track.y[i]+shift_x-radius), 
                   (t, target_track.x[i]+shift_y+radius, target_track.y[i]+shift_x+radius), 
                   (t, target_track.x[i]+shift_y-radius, target_track.y[i]+shift_x+radius))) 
                    for i,t in enumerate(range(target_track.t[0], target_track.t[-1]))
                    for radius in radii] 
    #return radius_shape
    return viewer.add_shapes(radius_shape, opacity = 1, shape_type = 'ellipse', face_color = 'transparent', edge_color = 'cyan', edge_width = 5, name = 'Radial environment')

def plot_stationary_radii(cell_type, target_track, radius, num_bins):
    print('This can be very time consuming for >10 bins, consider using single_frame radius')
    radii = range(int(radius/num_bins), radius+int(radius/num_bins), int(radius/num_bins))    
    if cell_type.lower() == 'scr':
        target_track = [track for track in scr_tracks if track.ID == cell_ID][0]
    else:
        target_track = [track for track in wt_tracks if track.ID == cell_ID][0]
    radius_shape = [tuple(((t, target_track.x[-1]+shift_y-radius, target_track.y[-1]+shift_x-radius), 
                   (t, target_track.x[-1]+shift_y+radius, target_track.y[-1]+shift_x-radius), 
                   (t, target_track.x[-1]+shift_y+radius, target_track.y[-1]+shift_x+radius), 
                   (t, target_track.x[-1]+shift_y-radius, target_track.y[-1]+shift_x+radius)))
                    for i,t in enumerate(range(target_track.t[-1]+1,len(gfp)))
                    for radius in radii] 
    #return radius_shape
    return viewer.add_shapes(radius_shape, opacity = 1, shape_type = 'ellipse', face_color = 'transparent', edge_color = 'cyan', edge_width = 5, name = 'Radial environment')

def plot_single_frame_radii(cell_type, target_track, radius, num_bins, frame):
    t = frame
    if cell_type.lower() == 'scr':
        target_track = [track for track in scr_tracks if track.ID == cell_ID][0]
    else:
        target_track = [track for track in wt_tracks if track.ID == cell_ID][0]

    try:
        i = target_track.t.index(t)
    except:
        i=-1
    radii = range(int(radius/num_bins), radius+int(radius/num_bins), int(radius/num_bins))    
    radius_shape = [tuple(((t, target_track.x[i]+shift_y-radius, target_track.y[i]+shift_x-radius), 
                   (t, target_track.x[i]+shift_y+radius, target_track.y[i]+shift_x-radius), 
                   (t, target_track.x[i]+shift_y+radius, target_track.y[i]+shift_x+radius), 
                   (t, target_track.x[i]+shift_y-radius, target_track.y[i]+shift_x+radius))) 
                    for radius in radii] 
    #return radius_shape
    return viewer.add_shapes(radius_shape, opacity = 1, shape_type = 'ellipse', face_color = 'transparent', edge_color = 'cyan', edge_width = 5, name = 'Radial environment')
   

# fixing fragmented display

In [10]:
# compiled_frag_track_loc = []
# compiled_frag_radius_loc = []
# for cell_ID in [17,42,47]:
#     target_track = [track for track in scr_tracks if track.ID == cell_ID][0]
#     #plot_radius(target_track)
#     #plot_target_track(target_track)
#     radius_loc = plot_frag_radius(target_track)
#     compiled_frag_radius_loc+= radius_loc
#     target_track_loc = plot_frag_target_track(target_track)
#     compiled_frag_track_loc += target_track_loc

### Launch napari to check against raw data 

In [20]:
with napari.gui_qt():
    viewer = napari.Viewer()
    
    ## add imagery
    viewer.add_image(bf, name = "brightfield", contrast_limits =(0,127))
    viewer.add_image(gfp, name="gfp", contrast_limits = (30, 180), blending = "additive", colormap = "green")
    viewer.add_image(rfp, name="rfp", contrast_limits = (4, 20), blending = "additive", colormap = "magenta")
    
    ## add tracks
    viewer.add_tracks(wt_data)
    viewer.add_tracks(scr_data)
    
    ## save out animations with widget
    animation_widget = AnimationWidget(viewer)
    viewer.window.add_dock_widget(animation_widget, area='right')
    
    ## save out movie frames (this method will take approx 10mins for 1200 frames)
    @viewer.bind_key('m')
    def make_movie(viewer):
        """Make a movie."""
        num_frames = len(gfp)-1#int(viewer.dims.range[0][1])
        def _make_movie():
            date = datetime.now().strftime("%Y_%m_%d-%I:%M:%S_%p")
            save_folder = f'/home/nathan/Documents/presentations/figures_animations/movie_{date}'
            os.mkdir(save_folder)
            for i in range(num_frames+1):
                viewer.dims.set_point(0, i)
                fn = os.path.join(save_folder, f'movie_{i}.png')
                image = viewer.screenshot(path=fn, canvas_only=True)
            print('Movie saved out')
        _make_movie()
    
    # plot annotations
#    plot_radii(17, 400, 10)
#     plot_radius(17, 400)
#     plot_stationary_radii(17, 400,10)
    plot_single_frame_radii('wt', 312, 400, 10, 591)
    
    
#     # plot each bin as own layer
#     radii = [40, 80, 120, 160, 200, 240, 280, 320, 360, 400]
#     for radius in radii:
#         plot_single_frame_radii(17, radius, 1, 730)